# Senate Elections, 1976-2020
This Notebook computes the tables required for the Senate Elections example dashboard.

Step 1: Import the libraries

In [ ]:
import pandas as pd
import json

Use Pandas to read the libraries, and view the result

In [ ]:
elections = pd.read_csv('data/1976-2020-senate.csv')

Cut out irrelevant columns, compute the percentage, and trim out candidates who got < 5% of the vote

In [ ]:
elections_trimmed = elections[['year', 'state', 'special', 'candidate', 'party_simplified', 'candidatevotes', 'totalvotes']]
elections_two = elections_trimmed.assign(percentage=pd.Series(elections_trimmed['candidatevotes']/elections_trimmed['totalvotes'] * 100))
elections_three = elections_two[elections_two['percentage'] > 5]

In [ ]:
elections_three

In [ ]:

def save_dataframe(dataframe, typelist, table_name):
    columns = dataframe.columns.tolist()
    assert len(columns) == len(typelist)
    schema  = [{"name": dataframe.columns[i], "type": typelist[i]} for i in range(len(typelist))]
    rows = dataframe.values.tolist()
    name = table_name if table_name.endswith('.sdml') else f'{table_name}.sdml'
    table =  {
        "type": "RowTable",
        "schema": schema,
        "rows": rows
    }

    with open(f'scratch/{name}', 'w') as f:
        json.dump(table, f, indent=2)
    

In [ ]:
typelist = ['number', 'string', 'boolean', 'string', 'string', 'number', 'number', 'number']
save_dataframe(elections_three, typelist,  'senate_vote')


Compute the history table, where the percentage of the vote by each party for each election is broken out as a separate column

In [ ]:


elections_by_party = elections_three.pivot_table(
    index=['year', 'state', 'special'],
    columns='party_simplified',
    values='percentage',
    fill_value=0  # Optional: fill missing with 0, or use None if you prefer
).reset_index()

# Optional: flatten the columns, if needed (if you get MultiIndex columns)
elections_by_party.columns.name = None
elections_by_party = elections_py_party.rename_axis(None, axis=1)

elections_by_party

In [ ]:
typelist  = ['number', 'string', 'boolean', 'nmumber', 'number', 'number', 'number']
save_dataframe(elections_by_party, typelist,  'senate_history')

Read the Senate control table (how many seats before/after each election) and publish it

In [ ]:
control = pd.read_csv('data/senate-control.csv')
save_dataframe(control, ["number" for column in control.columns.tolist()], 'senate-control')